In [1]:
import sys 
#sys.path.append('/home/monte.flora/python_packages/wofs_ml_severe')
sys.path.append('/home/monte.flora/python_packages/WoF_post')
#sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/experiments')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')
from ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset)
from os.path import join

In [2]:
dates = ['20190509']

In [3]:
##date = '20210504'

init_time = '1900' #3 hours out from effective time?

for date in dates:
    #path = f'/work/samuel.varga/data/2to6_hr_severe_wx/SummaryFiles/{date}/{init_time}'
    path = f'/work/mflora/SummaryFiles/{date}/{init_time}' #What files would I need to make it myself? Env and storm variables?
    out_path = join('/work/samuel.varga/data/2to6_hr_severe_wx', 'singleCase') 

    X_env, X_strm, ncfile, llgrid  = load_dataset(path)
    extracter = GridPointExtracter(ncfile, env_vars=X_env.keys(), strm_vars=X_strm.keys(), ll_grid=llgrid)
    df = extracter(X_env, X_strm, predict=True)
    df.reset_index(drop=True, inplace=True)

    out_name = join(out_path, f'wofs_ml_severe__2to6hr__{date}_data.feather')
    print(f'Saving {out_name}...')
    df.to_feather(out_name)

Saving /work/samuel.varga/data/2to6_hr_severe_wx/singleCase/wofs_ml_severe__2to6hr__20190509_data.feather...


# Trouble shooting steps

In [4]:
#path = f'/work/mflora/SummaryFiles/{dates[0]}/{init_time}'
#out_path = join('/work/samuel.varga/data/2to6_hr_severe_wx', 'singleCase')

In [5]:
#X_env, X_strm, ncfile, llgrid  = load_dataset(path)

In [6]:
#extracter = GridPointExtracter(ncfile, env_vars=X_env.keys(), strm_vars=X_strm.keys(), ll_grid=llgrid)
#df = extracter(X_env, X_strm, predict=True)
#df.reset_index(drop=True, inplace=True)

In [7]:
#for date in dates:
#    path = f'/work/mflora/SummaryFiles/{date}/{init_time}'
#    print(path)